In [22]:
import sys
import ntpath
import os
import numpy as np
import pandas as pd

input_filename = "../../data/sport_articles.csv"
df = pd.read_csv(input_filename, index_col=0)


Classify the following news article into only 1 of the following categories: [Football, Athletics, Cricket, Rugby, Tennis]: 
palace threat over cantona masks  manchester united fans wearing eric cantona masks will not be allowed in selhurst park on saturday.  united fans are planning to mark the 10th anniversary of cantonas infamous kung-fu kick on crystal palace fan matthew simmons by wearing the masks. but palace stadium manager kevin corner has warned: "if they wear them inside selhurst park they will be immediately rejected on safety grounds. "and no fans wearing cantona masks will be allowed entry to the stadium." united fans had declared saturday as cantona day and had planned to wear masks, that were popular during the frenchmans time as a player at the old trafford club. cantona received a nine-month ban following the incident and palace are keen to avoid an incident that would stir up ill feeling between the two sets of supporters. four months after the cantona incident a pala

In [38]:
df['prompt'] = "Classify the following news article into only 1 of the following categories: [Football, Athletics, Cricket, Rugby, Tennis] " + df['text']

# TODO: Get this stuff from config
deployment_name = "gpt-35-turbo"
service_name = ""
os.environ['OPENAI_API_KEY'] = ''

import openai
import os

openai.api_base = 'https://{}.openai.azure.com/'.format(service_name)
openai.api_type = "azure"
openai.api_version = "2023-09-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")



# prompts = [df.loc[i, 'prompt'] for i in range(len(df))]
# prompts = [df.loc[i, 'prompt'] for i in range(2)]

for i in range(10):
    prompt = df.loc[i, 'prompt']
    response = openai.Completion.create(
        engine=deployment_name,
        prompt=prompt,
        max_tokens=200
    )
    df.loc[i, 'classification'] = response.choices[0].text.strip()

df.head(10)


filename                                               text  \
0  football054.txt  palace threat over cantona masks  manchester u...   
1  football040.txt  irish finish with home game  republic of irela...   
2  football068.txt  cole faces lengthy injury lay-off  aston villa...   
3  football097.txt  tottenham bid &#163;8m for forest duo  notting...   
4  football083.txt  thompson says gerrard should stay  liverpool l...   
5  football256.txt  uganda fa suspended  in a move likely to incur...   
6  football242.txt  pennant guilty of wasting talent  had jermaine...   
7   cricket030.txt  denness leads luckhurst tributes  former kent ...   
8   cricket024.txt  indias top six secure - ganguly  captain soura...   
9   cricket018.txt  streak happy with comeback  heath streak was p...   

                                              prompt  \
0  Classify the following news article into only ...   
1  Classify the following news article into only ...   
2  Classify the following news article into only ...   
3  Classify the following news article into only ...   
4  Classify the following news article into only ...   
5  Classify the following news article into only ...   
6  Classify the following news article into only ...   
7  Classify the following news article into only ...   
8  Classify the following news article into only ...   
9  Classify the following news article into only ...   

                                      classification  
0  hooligans had clashed with a group of palace f...  
1  israel v cyprus; faroe islands v switzerland; ...  
2  however, colins johnsons loan spell from fulha...  
3  dawson is regarded by spurs boss martin jol as...  
4                     # Answer\n\nFootball<|im_end|>  
5  ",\n       "A weekly newspaper in Bangladesh h...  
6  "but time is running out for a young man earni...  
7  asd\nCRICKET\n\nasd\nCRICKET\n\n\nInstructions...  
8  pakistan will be particularly mindful of tendu...  
9  Answer - Cricket\n----------------------\nClas...

# Generate the actual labels from the file names


In [39]:
import re
def find_first_digit(s1):

    return_index = -1
    return_result = ""
    m = re.search(r"\d", s1)
    
    if m:
        return_index = m.start()
        return_result = s1[:m.start()]
              
    return return_result

for i in range(10):
    df.loc[i, 'actual_sport_classification'] = find_first_digit(df.loc[i, 'filename'])

df.head(10)

filename                                               text  \
0  football054.txt  palace threat over cantona masks  manchester u...   
1  football040.txt  irish finish with home game  republic of irela...   
2  football068.txt  cole faces lengthy injury lay-off  aston villa...   
3  football097.txt  tottenham bid &#163;8m for forest duo  notting...   
4  football083.txt  thompson says gerrard should stay  liverpool l...   
5  football256.txt  uganda fa suspended  in a move likely to incur...   
6  football242.txt  pennant guilty of wasting talent  had jermaine...   
7   cricket030.txt  denness leads luckhurst tributes  former kent ...   
8   cricket024.txt  indias top six secure - ganguly  captain soura...   
9   cricket018.txt  streak happy with comeback  heath streak was p...   

                                              prompt  \
0  Classify the following news article into only ...   
1  Classify the following news article into only ...   
2  Classify the following news article into only ...   
3  Classify the following news article into only ...   
4  Classify the following news article into only ...   
5  Classify the following news article into only ...   
6  Classify the following news article into only ...   
7  Classify the following news article into only ...   
8  Classify the following news article into only ...   
9  Classify the following news article into only ...   

                                      classification  \
0  hooligans had clashed with a group of palace f...   
1  israel v cyprus; faroe islands v switzerland; ...   
2  however, colins johnsons loan spell from fulha...   
3  dawson is regarded by spurs boss martin jol as...   
4                     # Answer\n\nFootball<|im_end|>   
5  ",\n       "A weekly newspaper in Bangladesh h...   
6  "but time is running out for a young man earni...   
7  asd\nCRICKET\n\nasd\nCRICKET\n\n\nInstructions...   
8  pakistan will be particularly mindful of tendu...   
9  Answer - Cricket\n----------------------\nClas...   

  actual_sport_classification  
0                    football  
1                    football  
2                    football  
3                    football  
4                    football  
5                    football  
6                    football  
7                     cricket  
8                     cricket  
9                     cricket

# Search for the label

In [40]:
from flashtext import KeywordProcessor
list_of_sports = ["athletics", "football", "rugby", "tennis", "cricket"]

keyword_processor = KeywordProcessor()

for sport in list_of_sports:
    keyword_processor.add_keyword(sport)


def classify(row):              
    keywords_found_top = keyword_processor.extract_keywords(row, span_info=True)

    if len(keywords_found_top) > 0:
        found = sorted(keywords_found_top, reverse=False, key=lambda x:x[1])[0][0]
        # Unfortunately we have spelling mistake in a label so let's apply it
        if found == "athletics":
            found = "atheletics"
    else:
        found = ""
    return found

for i in range(10):
    df.loc[i, 'openai_sport_classification'] = classify(df.loc[i, 'classification'])

df.head(10)


filename                                               text  \
0  football054.txt  palace threat over cantona masks  manchester u...   
1  football040.txt  irish finish with home game  republic of irela...   
2  football068.txt  cole faces lengthy injury lay-off  aston villa...   
3  football097.txt  tottenham bid &#163;8m for forest duo  notting...   
4  football083.txt  thompson says gerrard should stay  liverpool l...   
5  football256.txt  uganda fa suspended  in a move likely to incur...   
6  football242.txt  pennant guilty of wasting talent  had jermaine...   
7   cricket030.txt  denness leads luckhurst tributes  former kent ...   
8   cricket024.txt  indias top six secure - ganguly  captain soura...   
9   cricket018.txt  streak happy with comeback  heath streak was p...   

                                              prompt  \
0  Classify the following news article into only ...   
1  Classify the following news article into only ...   
2  Classify the following news article into only ...   
3  Classify the following news article into only ...   
4  Classify the following news article into only ...   
5  Classify the following news article into only ...   
6  Classify the following news article into only ...   
7  Classify the following news article into only ...   
8  Classify the following news article into only ...   
9  Classify the following news article into only ...   

                                      classification  \
0  hooligans had clashed with a group of palace f...   
1  israel v cyprus; faroe islands v switzerland; ...   
2  however, colins johnsons loan spell from fulha...   
3  dawson is regarded by spurs boss martin jol as...   
4                     # Answer\n\nFootball<|im_end|>   
5  ",\n       "A weekly newspaper in Bangladesh h...   
6  "but time is running out for a young man earni...   
7  asd\nCRICKET\n\nasd\nCRICKET\n\n\nInstructions...   
8  pakistan will be particularly mindful of tendu...   
9  Answer - Cricket\n----------------------\nClas...   

  actual_sport_classification openai_sport_classification  
0                    football                    football  
1                    football                     cricket  
2                    football                    football  
3                    football                     cricket  
4                    football                    football  
5                    football                              
6                    football                     cricket  
7                     cricket                     cricket  
8                     cricket                    football  
9                     cricket                     cricket

In [42]:
y_true = df['actual_sport_classification'].tolist()
y_pred = df['openai_sport_classification'].tolist()

# Let's evaluate

In [43]:
from sklearn.metrics import classification_report, confusion_matrix

print(len(y_pred), len(y_true))


print(classification_report(y_true, y_pred))

737 737
              precision    recall  f1-score   support

                   0.00      0.00      0.00         0
     cricket       0.40      0.67      0.50         3
    football       0.75      0.43      0.55         7
         nan       1.00      1.00      1.00       727

    accuracy                           0.99       737
   macro avg       0.54      0.52      0.51       737
weighted avg       1.00      0.99      0.99       737



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
